## Simple GenAI APP Using LangChain

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ['OPENAI_API_KEY']= os.getenv("OPENAI_API_KEY")

In [ ]:

os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")

os.environ["LANGCHAIN_TRACING_V2"]= "true"

#### Load Data-->Docs-->Divide our Documents into Chunks(text)-->(text-->Vector)Vector Embeddings--->VectorStoreDB

## Data Ingestion--
#### From the website we need to scrape the data

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader= WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [ ]:
docs= loader.load()
docs

## Splitting

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents= text_splitter.split_documents(docs)

In [ ]:
documents

## Embeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings= OpenAIEmbeddings()

## VectorStore DB

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstoredb= FAISS.from_documents(documents, embeddings)

In [ ]:
vectorstoredb

## Query From a vector db

In [ ]:
query= "LangSmith has two usage limits: total traces and extended"

result= vectorstoredb.similarity_search(query)
result

In [ ]:
result[0].page_content

## Documents Chain

#### Using LLM

In [ ]:
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(model="gpt-4o")

#### Building Prompt

In [ ]:

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_template(
    """
            Answer the following question based only on the provided context:
            <context>
            {context}
            </context>

    """
)

document_chain= create_stuff_documents_chain(llm, prompt)
document_chain

#### Using the Document chain

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content= "LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

In [ ]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

## Retrieval Chain

In [ ]:
retriever= vectorstoredb.as_retriever()

from langchain.chains import create_retrieval_chain

retrieval_chain= create_retrieval_chain(retriever, document_chain)


In [ ]:
retrieval_chain

## Get the response from the LLM

In [ ]:

response= retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

In [ ]:

response

In [ ]:
response['context']